In [1]:
# G. van Rossum, Python tutorial, Technical Report CS-R9526, 
# Centrum voor Wiskunde en Informatica (CWI), Amsterdam, May 1995." 
from platform import python_version
print(python_version())

# Fernando Pérez, Brian E. Granger, IPython: A System for Interactive Scientific Computing, 
# Computing in Science and Engineering, vol. 9, no. 3, pp. 21-29, May/June 2007, 
# doi:10.1109/MCSE.2007.53. URL: https://ipython.org
import IPython
print(IPython.__version__)

%autosave 120

2.7.15
5.8.0


Autosaving every 120 seconds


Get data from drought website by climate division:  
https://www.ncdc.noaa.gov/cag/national/time-series/110/pdsi/1/6/1895-2018?base_prd=true&firstbaseyear=1901&lastbaseyear=2000

The website data comes from the U.S. Climate Divisional Database:  
https://www.ncdc.noaa.gov/monitoring-references/maps/us-climate-divisions.php

The current dataset (nClimDiv) can be downloaded at:  
ftp://ftp.ncdc.noaa.gov/pub/data/cirs/climdiv/

In [2]:
import csv

# Downloaded climdiv-pdsidv-v1-0-0-20180705.txt from ftp://ftp.ncdc.noaa.gov/pub/data/cirs/climdiv/
# See ftp://ftp.ncdc.noaa.gov/pub/data/cirs/climdiv/drought-readme.txt for details.

# Vose, Russell S.; Applequist, Scott; Squires, Mike; Durre, Imke; Menne, Matthew J.; Williams, Claude N., Jr.;
# Fenimore, Chris; Gleason, Karin; Arndt, Derek (2014): NOAA's Gridded Climate Divisional Dataset (CLIMDIV). 
# [indicate subset used]. NOAA National Climatic Data Center. doi:10.7289/V5M32STR [access date].

with open('pdsi_data\climdiv-pdsidv-v1-0-0-20180705.txt', 'r') as inf:
    with open('pdsi_data\climdiv-pdsidv.csv', 'wb') as outf:
        header = ['STATE-CODE','DIVISION-NUMBER','ELEMENT-CODE','YEAR','JAN-VALUE','FEB-VALUE','MAR-VALUE','APR-VALUE',\
                 'MAY-VALUE','JUNE-VALUE','JULY-VALUE','AUG-VALUE','SEPT-VALUE','OCT-VALUE','NOV-VALUE','DEC-VALUE']
        writer = csv.DictWriter(outf, fieldnames=header)
        writer.writeheader()
        for row in inf:
            record = {
                'STATE-CODE': row[0:2],
                'DIVISION-NUMBER': row[2:4],
                'ELEMENT-CODE': row[4:6],
                'YEAR': row[6:10],
                'JAN-VALUE': row[10:17].lstrip(),
                'FEB-VALUE': row[17:24].lstrip(),
                'MAR-VALUE': row[24:31].lstrip(),
                'APR-VALUE': row[31:38].lstrip(),
                'MAY-VALUE': row[38:45].lstrip(),
                'JUNE-VALUE': row[45:52].lstrip(),
                'JULY-VALUE': row[52:59].lstrip(),
                'AUG-VALUE': row[59:66].lstrip(),
                'SEPT-VALUE': row[66:73].lstrip(),
                'OCT-VALUE': row[73:80].lstrip(),
                'NOV-VALUE': row[80:87].lstrip(),
                'DEC-VALUE': row[87:94].strip(),
            }
            writer.writerow(record)

In [3]:
# Wes McKinney. Data Structures for Statistical Computing in Python, 
# Proceedings of the 9th Python in Science Conference, 51-56 (2010) (publisher link)
import pandas as pd

pd.options.display.max_rows = 6
pd.options.display.precision = 3

# df <- read.csv("data/climdiv-pdsidv.csv",
#                 colClasses = c(DIVISION.NUMBER='character')) %>%
#       mutate(STDIV = str_c(STATE.CODE, "0", DIVISION.NUMBER)) %>%
#       select(STDIV, YEAR, ends_with('VALUE')) %>%
#       filter(df$YEAR >= 1968) %>%
#       melt(id.vars = c('STDIV', 'YEAR'), value.name = 'PDSI')

df = pd.read_csv('pdsi_data/climdiv-pdsidv.csv')
df['STDIV'] = df['STATE-CODE'].astype(str) + df['DIVISION-NUMBER'].astype(str).str.pad(3, side='left', fillchar='0')
cols = [col for col in df.columns if col.endswith('VALUE')]
df = pd.melt(df, id_vars=['STDIV', 'YEAR'], value_vars=cols, var_name='MONTH', value_name='PDSI')
df = df.replace(to_replace={col: i+1 for i, col in enumerate(cols)})
df = df.loc[(df.YEAR >= 1968) & (df.YEAR < 2018) & (df.STDIV.astype(int) <= 48999)]
df

,STDIV,YEAR,MONTH,PDSI
73,1001,1968,1,3.66
74,1001,1969,1,-1.11
75,1001,1970,1,-0.67
...,...,...,...,...
555516,48010,2015,12,0.15
555517,48010,2016,12,0.30
555518,48010,2017,12,-1.56


 Calculate means and sd for 1,3,5,10,15 yrs.

In [4]:
REVISION = 20180727

end_year = 2016
intervals = [1, 3, 5, 10, 15]
start_year = [(end_year + 1) - i for i in intervals]

summary = pd.DataFrame(df.STDIV.unique()).rename(columns={0: 'STDIV'})

for start in start_year:

    period = range(start, end_year+1)

    mean = df.loc[df.YEAR.isin(period)].groupby('STDIV').PDSI.mean().round(2)
    mean = mean.to_frame().rename(columns={'PDSI': 'PDSI_MEAN_{}'.format(start)})

    std = df.loc[df.YEAR.isin(period)].groupby('STDIV').PDSI.std().round(2)
    std = std.to_frame().rename(columns={'PDSI': 'PDSI_STD_{}'.format(start)})

    temp = mean.merge(std, on='STDIV').reset_index()
    summary = summary.merge(temp, on='STDIV')

summary

,STDIV,PDSI_MEAN_2016,PDSI_STD_2016,PDSI_MEAN_2014,PDSI_STD_2014,PDSI_MEAN_2012,PDSI_STD_2012,PDSI_MEAN_2007,PDSI_STD_2007,PDSI_MEAN_2002,PDSI_STD_2002
0,1001,-1.47,1.00,0.30,1.59,0.27,1.55,-0.47,2.10,-0.20,1.99
1,1002,-2.22,1.53,-0.72,1.52,0.02,1.85,-0.52,2.28,-0.29,2.12
2,1003,-1.44,1.23,-0.24,1.32,0.00,1.47,-0.53,2.15,0.02,2.19
...,...,...,...,...,...,...,...,...,...,...,...
341,48008,0.69,2.40,2.08,1.75,-0.16,3.88,0.15,3.40,-1.12,3.40
342,48009,0.13,1.39,0.30,1.38,-1.21,2.98,-0.97,2.92,-1.82,2.93
343,48010,-0.16,1.24,0.19,1.22,-1.32,3.14,-0.40,3.54,-1.62,3.56


In [5]:
# summary.to_csv('pdsi_data/pdsi_summary_{}.csv'.format(REVISION), index=False)

Calculate the deviation from the mean (established over the previous 30 years) for each climate division  
for the time-steps: 2016 (1 year); 2014 (3 years); 2012 (5 years); 2007 (10 years); and 2002 (15 years).

In [6]:
REVISION = 20190131

summary = pd.DataFrame(df.STDIV.unique()).rename(columns={0: 'STDIV'})

for year in [2016, 2014, 2012, 2007, 2002]:

    mean_30yr = (df.loc[df.YEAR.isin(range(year-31, year))]
                   .groupby('STDIV')
                   .PDSI.mean())

    mean_yr = (df.loc[df.YEAR == year]
             .groupby('STDIV')
             .PDSI.mean())

    deviation = (mean_30yr.subtract(mean_yr)
                          .to_frame()
                          .rename(columns={'PDSI': 'PDSI_DEV_{}'.format(year)})
                          .reset_index()
                          .round(2))
    
    summary = summary.merge(deviation, on='STDIV')
    
summary

,STDIV,PDSI_DEV_2016,PDSI_DEV_2014,PDSI_DEV_2012,PDSI_DEV_2007,PDSI_DEV_2002
0,1001,1.65,-0.72,0.63,4.31,-0.82
1,1002,2.18,-0.12,0.09,4.35,-0.14
2,1003,1.46,-0.56,0.93,4.00,0.07
...,...,...,...,...,...,...
341,48008,-1.12,-3.11,3.90,3.34,5.39
342,48009,-1.22,-1.80,3.16,4.67,6.66
343,48010,-0.82,-1.63,3.06,3.83,7.47


In [7]:
summary.to_csv('pdsi_data/pdsi_summary_{}.csv'.format(REVISION), index=False)